In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = pd.read_csv('fake_or_real_news.csv')

In [3]:
df.head()

id                                              title  \
0   8476                       You Can Smell Hillary’s Fear   
1  10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2   3608        Kerry to go to Paris in gesture of sympathy   
3  10142  Bernie supporters on Twitter erupt in anger ag...   
4    875   The Battle of New York: Why This Primary Matters   

                                                text label  
0  Daniel Greenfield, a Shillman Journalism Fello...  FAKE  
1  Google Pinterest Digg Linkedin Reddit Stumbleu...  FAKE  
2  U.S. Secretary of State John F. Kerry said Mon...  REAL  
3  — Kaydee King (@KaydeeKing) November 9, 2016 T...  FAKE  
4  It's primary day in New York and front-runners...  REAL

In [4]:
df['fake'] = df.label.apply(lambda x : 1 if x == 'FAKE' else 0)

In [5]:
df.drop('label' , axis=1 , inplace=True)

In [6]:
df.head()

id                                              title  \
0   8476                       You Can Smell Hillary’s Fear   
1  10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2   3608        Kerry to go to Paris in gesture of sympathy   
3  10142  Bernie supporters on Twitter erupt in anger ag...   
4    875   The Battle of New York: Why This Primary Matters   

                                                text  fake  
0  Daniel Greenfield, a Shillman Journalism Fello...     1  
1  Google Pinterest Digg Linkedin Reddit Stumbleu...     1  
2  U.S. Secretary of State John F. Kerry said Mon...     0  
3  — Kaydee King (@KaydeeKing) November 9, 2016 T...     1  
4  It's primary day in New York and front-runners...     0

In [7]:
X_text , X_title , y = df.text , df.title , df.fake

In [8]:
vec_text = TfidfVectorizer(stop_words='english' , max_df=0.7)
vec_title = TfidfVectorizer(stop_words='english' , max_df=0.7)

X_text = vec_text.fit_transform(X_text)
X_title = vec_title.fit_transform(X_title)

In [11]:
from sklearn.ensemble import RandomForestRegressor

In [12]:
def test(model , X , y , split):
    X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = split)
    model = model.fit(X_train , y_train)
    return model , model.score(X_test , y_test) ,model.score(X_train , y_train)

In [13]:
model_text , test_acc_text , train_acc_text = test(RandomForestRegressor() , X_text , y , 0.1)
model_title , test_acc_title , train_acc_title = test(RandomForestRegressor() , X_title , y , 0.1)

In [14]:
import tensorflow as tf

def train_nn(model_text , model_title , X_title , X_text , y , split):
    df = pd.DataFrame()
    df['text'] = model_text.predict(X_text)
    df['title'] = model_title.predict(X_title)
    X_train , X_test , y_train , y_test = train_test_split(df , y , test_size = split)
    input_shape = [X_train.shape[1]]
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=2, activation='relu',input_shape=input_shape),
    tf.keras.layers.Dense(units=1,activation='sigmoid')])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train , y_train , epochs=15)
    return model , model.evaluate(X_test , y_test)

In [15]:
model , score = train_nn(model_text , model_title , X_title , X_text , y , 0.2)
print(score)

Epoch 1/15
159/159 [==============================] - 2s 5ms/step - loss: 0.6932 - accuracy: 0.4939
Epoch 2/15
159/159 [==============================] - 1s 5ms/step - loss: 0.6931 - accuracy: 0.4937
Epoch 3/15
159/159 [==============================] - 1s 4ms/step - loss: 0.6931 - accuracy: 0.4957
Epoch 4/15
159/159 [==============================] - 1s 4ms/step - loss: 0.6930 - accuracy: 0.4856
Epoch 5/15
159/159 [==============================] - 1s 4ms/step - loss: 0.5866 - accuracy: 0.7342
Epoch 6/15
159/159 [==============================] - 1s 4ms/step - loss: 0.4387 - accuracy: 0.9201
Epoch 7/15
159/159 [==============================] - 1s 4ms/step - loss: 0.3559 - accuracy: 0.9619
Epoch 8/15
159/159 [==============================] - 1s 4ms/step - loss: 0.2983 - accuracy: 0.9765
Epoch 9/15
159/159 [==============================] - 1s 4ms/step - loss: 0.2552 - accuracy: 0.9826
Epoch 10/15
159/159 [==============================] - 1s 4ms/step - loss: 0.2217 - accuracy: 0.9868

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2)                 6         
                                                                 
 dense_1 (Dense)             (None, 1)                 3         
                                                                 
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.predict([[model_text.predict(vec_text.transform(['This is a fake new to test the model']))[0] ,
                model_title.predict(vec_title.transform(['fake model']))[0]]])

1/1 [==============================] - 0s 99ms/step


array([[0.9694139]], dtype=float32)